# Trivial Analytics
The purpose of this experiment is to answer this question:

> If you only could study 100 topics in preparation for Jeopardy!, which topics should you study

Hopefully this will give me some practice doing data analytics on a relatively small data set, give me some insight into something I am interested in, and expose me to some natural language processing topics.

I found a reddit poster sharing a data set with 200,000+ questions here https://www.reddit.com/r/datasets/comments/1uyd0t/200000_jeopardy_questions_in_a_json_file/

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="service",
  password="jeopardy!",
)

cursor = mydb.cursor()
cursor.execute("SELECT question, answer FROM jeapordy_questions.question WHERE question like '%Egypt%'")

for i in range(0, 10):
    row = cursor.fetchone()
    print("Question: | " + row[0])
    print("Answer:   | " + row[1])
    print()

## Named Entity Recognition
So what do I mean by 'topic'. The subject of a question could be pretty broad or pretty granular. Clearly it doesn't give us enough info on what to study just looking at the category of the question. Categories like 'history' are way too broad to be useful. Meanwhile many of the Jeapordy! categories are unique to the game, playful rhymes or word games.

In general, it looks like we are trying to extract people, places, times, etc. In NLP there is a name for annotating this type of information, 'Named Entity Recognition'. Fortunately there are handy python libraries out there like spaCy that can do the heavy lifting for this. https://spacy.io/api/entityrecognizer


In [41]:
# Reference https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

q = '"The Prince of Egypt" featured Ralph Fiennes as the voice of this stubborn ruler'

doc = nlp(q)
print([(X.text, X.label_) for X in doc.ents])

[('The Prince of Egypt', 'WORK_OF_ART'), ('Ralph Fiennes', 'PERSON')]


That's great, so we can put in a string and spaCy can help us identify the named entities.

The information that is useful is most often in the body of the question or the answer.

For example, "Galileo was sentenced to home arrest after supporting the theories of this astronomer" is a question about Copernicus, which appears in the answer. Galileo is also a useful piece of information, if you knew a lot about Galielo, you probably could get to Copernicus.

The main subject of the question could appear in the question as well. "Copernicus was prosecuted by the church for publishing a paper on this model of the solar system". Copernicus is still the main topic of the question, even though he doesn't appear in the answer, 'heliocentric'. 

Seems like it would be worthwhile to create a new table for named entities and then a mapping table to map q/a combinations that contain those named entities.


## Mapping Questions to Named Entities
To map question and answer text to Named Entities, we need a new table to track those entities and their types, as well as a mapping table to handle the many to many relationship of question to named_entity. With that set up and foreign keys in place, I should be able to populate those tables pretty easily.

This seeding script reads questions and answers from the database, creates named enitities and maps them to questions by inserting rows in named_entitu and mapping rows in question_named_entity. For now it is ignoring CARDINAL and MONEY named entities, as I found them to be not so useful (examples: 100, $1 billing, etc.).

In [19]:
import mysql.connector
import regex
import unidecode
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

mydb = mysql.connector.connect(
  host="localhost",
  user="service",
  password="jeopardy!",
)

# Queries
get_all_questions = ("SELECT question_id, question, answer FROM jeapordy_questions.question LIMIT %s OFFSET %s")
delete_mappings = "DELETE FROM jeapordy_questions.question_named_entity"
delete_named_entities = "DELETE FROM jeapordy_questions.named_entity"
add_named_entity = ("INSERT INTO jeapordy_questions.named_entity (name, type) VALUES (%s, %s)")
add_mapping = ("INSERT INTO jeapordy_questions.question_named_entity (question, named_entity) VALUES (%s, %s)")

cursor = mydb.cursor()
print("Deleting old records...")
cursor.execute(delete_mappings)
cursor.execute(delete_named_entities)
mydb.commit()

limit = 10000
offset = 0
data_entities = dict()
all_data_entities = dict()
data_mappings = []

for i in range(limit, 200000, limit):
  print("Starting get...")
  get_data = (limit, offset)
  print(str(get_data))
  cursor.execute(get_all_questions, get_data)

  print("Done, starting mapping...")
  for (question_id, question, answer) in cursor:
      q = unidecode.unidecode(regex.sub("'", "", question + " " + answer))
      doc = nlp(q)
      for X in doc.ents:
        if not X.label_ == 'CARDINAL' and not X.label_ == 'MONEY':
          entity = X.text.lower()
          if not entity in data_entities:
            if not entity in all_data_entities:
              data_entities[entity] = X.label_
          data_mapping = (question_id, entity)
          data_mappings.append(data_mapping)
  print("Done, starting insert...")
  cursor.executemany(add_named_entity, (list(data_entities.items())))
  mydb.commit()
  cursor.executemany(add_mapping, (data_mappings))
  mydb.commit()

  all_data_entities = {**all_data_entities, **data_entities} 
  data_entities.clear()
  data_mappings = []
  offset = i
print("Done, closing...")
cursor.close()

Deleting old records...
Starting get...
(10000, 0)
Done, starting mapping...
Done, starting insert...
Starting get...
(10000, 10000)
Done, starting mapping...
Done, starting insert...
Starting get...
(10000, 20000)
Done, starting mapping...
Done, starting insert...
Starting get...
(10000, 30000)
Done, starting mapping...
Done, starting insert...
Starting get...
(10000, 40000)
Done, starting mapping...
Done, starting insert...
Starting get...
(10000, 50000)
Done, starting mapping...
Done, starting insert...
Starting get...
(10000, 60000)
Done, starting mapping...
Done, starting insert...
Starting get...
(10000, 70000)
Done, starting mapping...
Done, starting insert...
Starting get...
(10000, 80000)
Done, starting mapping...
Done, starting insert...
Starting get...
(10000, 90000)
Done, starting mapping...
Done, starting insert...
Starting get...
(10000, 100000)
Done, starting mapping...
Done, starting insert...
Starting get...
(10000, 110000)
Done, starting mapping...
Done, starting inse

True

With that in place, we should be able to query the mapping table for information about occurrences of certain named entities in questions. We should be able to write a rudimentary query to answer our question- what are the most important topics to study?

In [33]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="service",
  password="jeopardy!",
)

cursor = mydb.cursor()
cursor.execute("SELECT named_entity,COUNT(*) FROM jeapordy_questions.question_named_entity GROUP BY named_entity ORDER BY COUNT(*) DESC LIMIT 10;")

for r in cursor:
    print(str(r))
cursor.close()

('first', 7848)
('u.s.', 3895)
('french', 2251)
('british', 1687)
('greek', 1449)
('american', 1388)
('latin', 1386)
('english', 1068)
('second', 1004)
('german', 957)


True

We're getting closer. Not too surprisingly, we see frequent occurences of what look like geographic or linquistic designations- French, American, English, Greek, German. First and Second don't really seem like topics, but more like just a part of speech, let's look into that.

In [42]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="service",
  password="jeopardy!",
)

cursor = mydb.cursor()
cursor.execute(
    "SELECT question.question, named_entity.type FROM jeapordy_questions.question_named_entity " + 
    "LEFT JOIN jeapordy_questions.question " +
    "ON jeapordy_questions.question.question_id = jeapordy_questions.question_named_entity.question " +
    "LEFT JOIN jeapordy_questions.named_entity " +
    "ON jeapordy_questions.question_named_entity.named_entity = jeapordy_questions.named_entity.name " +
    "WHERE jeapordy_questions.question_named_entity.named_entity = 'first' LIMIT 30;")

for r in cursor:
    print(str(r))

cursor.close()

("'Cows regurgitate this from the first stomach to the mouth & chew it again'", 'ORDINAL')
("'Karl led the first of these Marxist organizational efforts; the second one began in 1889'", 'ORDINAL')
('\'This "Modern Girl" first hit the Billboard Top 10 with "Morning Train (Nine To Five)"\'', 'ORDINAL')
("'Warhol became the manager of this Lou Reed rock group in 1965 & produced their first album'", 'ORDINAL')
("'His first act after being sworn in as president of the Confederacy was to send a peace commission to Washington, D.C.'", 'ORDINAL')
("'The first 50-star U.S. flag was officially raised on July 4 of this year'", 'ORDINAL')
('\'On March 19, 2009 he said, "I\'m excited and honored to introduce my first guest... Barack Obama"\'', 'ORDINAL')
("'The first controlled nuclear chain reaction'", 'ORDINAL')
('\'He reviewed films & TV for the New Republic before his first book, "Goodbye, Columbus", was published in 1959\'', 'ORDINAL')
("'Colo was the first of these great apes born in captivit

True

That looks like enough information to rule out ORDINAL. Out of curiosity lets repeat this process for things like American, or German. Maybe there is another broad category to rule out.

In [46]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="service",
  password="jeopardy!",
)

cursor = mydb.cursor()
cursor.execute(
    "SELECT question.question, named_entity.type FROM jeapordy_questions.question_named_entity " + 
    "LEFT JOIN jeapordy_questions.question " +
    "ON jeapordy_questions.question.question_id = jeapordy_questions.question_named_entity.question " +
    "LEFT JOIN jeapordy_questions.named_entity " +
    "ON jeapordy_questions.question_named_entity.named_entity = jeapordy_questions.named_entity.name " +
    "WHERE jeapordy_questions.question_named_entity.named_entity = 'german' LIMIT 30;")

for r in cursor:
    print(str(r))

cursor.close()

("'In 1811 this German family began its steel-making business by constructing a plant in Essen'", 'NORP')
("'In 1905 German scientist Alfred Einhorn created this first injectable local anesthetic used in dentistry'", 'NORP')
('\'Some of these are produced by bremsstrahlung, from the German for "breaking radiation"\'', 'NORP')
("'Named for a German neuropathologist, this memory loss disease may be caused by a gene on chromosome 21'", 'NORP')
("'This German composer's 5th Symphony in C Minor has a famous opening'", 'NORP')
("'In German, berg is this topographical feature on a map'", 'NORP')
("'The U.S. U-2, first built in the 1950s, was an airplane; the German U-1, first built in the 1910s, was one of these'", 'NORP')
("'Paul Baumer, a young German soldier'", 'NORP')
("'A German circus performer has made the Guinness record book for riding a bicycle with this distinction'", 'NORP')
("'Laboratory culture dish named for the German bacteriologist who invented it'", 'NORP')
('\'In "Sahara", 

True

Seems like NORP might be worth ruling out also. At this point I am starting to wonder if it is less about NLP labels we don't care about, and more about the few we DO care about. Interestingly, the first appearance of a topic that feels "trivial" in nature is "The Clue Crew" with 357 occurrences in questions. Let's take a look at those results and see how it is labeled. "The Clue Crew", obviously the group from the Nancy Drew childrens book series, is labeld as an "ORG". Logically seems like if we care about organizations, we also care about people.

In [52]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="service",
  password="jeopardy!",
)

cursor = mydb.cursor()
cursor.execute(
    "SELECT named_entity.name, COUNT(*) FROM jeapordy_questions.question_named_entity " + 
    "LEFT JOIN jeapordy_questions.named_entity " +
    "ON jeapordy_questions.question_named_entity.named_entity = jeapordy_questions.named_entity.name " +
    "WHERE jeapordy_questions.named_entity.type = 'ORG' OR jeapordy_questions.named_entity.type = 'PERSON'" +
    "GROUP BY named_entity ORDER BY COUNT(*) DESC LIMIT 10;")

for r in cursor:
    print(str(r))

cursor.close()

('congress', 388)
('nyc', 359)
('the clue crew', 357)
('shakespeare', 287)
('jesus', 274)
('senate', 257)
('sarah', 232)
('house', 220)
('nba', 219)
('john', 210)


True

By merging our two queries together, paying attention to occurrences of entities by name as long as their label is either a person or an organization, we can quickly see that a student of the game (or any well-rounded individual I suppose) would benefit by reading up on the U.S. Congress, the city of new york, Nancy Drew, Shakespeare, and Jesus.